In [10]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
csv_file = "data/stockholm-st-eriksgatan-83-air-quality.csv"
util.check_file_path(csv_file)

File successfully found at the path: data/stockholm-st-eriksgatan-83-air-quality.csv


,date,pm25,pm10,no2
0,2024/11/1,9,8,5
1,2024/11/2,9,5,4
2,2024/11/3,7,13,9
3,2024/11/4,11,7,6
4,2024/11/5,16,7,5
...,...,...,...,...
2195,2019/11/21,,18,8
2196,2019/11/23,,22,5
2197,2018/12/31,,8,4
2198,2018/4/8,,36,18


In [27]:
country = "sweden"
city = "stockholm"
street = "stockholm-st-eriksgatan-83"
aqicn_url="https://api.waqi.info/feed/@10523"

today = datetime.date.today()
latitude, longitude = util.get_city_coordinates(city)
print("Latitude: ", latitude, "Longitude: ", longitude)

Latitude:  59.33 Longitude:  18.07


In [28]:
import dotenv

dotenv.load_dotenv()

AQI_API_KEY = os.getenv("AQI_API_KEY")
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

project = hopsworks.login()

secrets = util.secrets_api(project.name)
try:
    secrets.create_secret("AQI_API_KEY", AQI_API_KEY)
except hopsworks.RestAPIError:
    AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157270
Connected. Call `.close()` to terminate connection gracefully.


In [29]:
try:
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
except hopsworks.RestAPIError:
    print("It looks like the AQI_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

aq_today_df.head()

,pm25,country,city,street,date,url
0,16.0,sweden,stockholm,stockholm-st-eriksgatan-83,2024-11-12,https://api.waqi.info/feed/@10523


In [ ]:
df = pd.read_csv(csv_file, parse_dates=["date"], skipinitialspace=True)

df = df[["date", "pm25"]]
df["country"] = country
df["city"] = city
df["street"] = street
df["url"] = aqicn_url

df["pm"]
df

,date,pm25,country,city,street,url
0,2024-11-01,9.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
1,2024-11-02,9.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
2,2024-11-03,7.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
3,2024-11-04,11.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
4,2024-11-05,16.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
...,...,...,...,...,...,...
2195,2019-11-21,NaN,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
2196,2019-11-23,NaN,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
2197,2018-12-31,NaN,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
2198,2018-04-08,NaN,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523
